<a href="https://colab.research.google.com/github/profteachkids/STEMUnleashed2023/blob/main/tSNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
!pip install datasets
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.3 MB/s eta 0:00:00


In [260]:
import jax.numpy as jnp
import jax
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
from einops import rearrange
from scipy.optimize import minimize

In [105]:
from datasets import load_dataset

dataset = load_dataset("mnist", split="train")

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset mnist downloaded and prepared to /root/.cache/huggingface/datasets/mnist/mnist/1.0.0/9d494b7f466d6931c64fb39d58bb1249a4d85c9eb9865d9bc20960b999e2a332. Subsequent calls will reuse this data.


In [180]:
N=300
imgs=np.stack(dataset[:N]['image'])
labels=dataset[:N]['label']

In [181]:
plot_img=rearrange(imgs[:9], '(b1 b2) h w -> (b1 h) (b2 w)', b1=3)

In [182]:
fig=px.imshow(plot_img, color_continuous_scale='gray')
fig.update_layout(width=300,height=300,template='plotly_dark',coloraxis_showscale=False)
fig.show()

In [183]:
x=rearrange(imgs, 'b h w -> b (h w)').astype(np.float64)

In [255]:

twosigmasq = np.mean(np.sum((x[:,None,:]-x[None,:,:])**2,axis=-1))/5
expnegdelta=np.exp(-np.sum((x[:,None,:]-x[None,:,:])**2,axis=-1)/twosigmasq)+1e-15
np.fill_diagonal(expnegdelta,0.)
pij=expnegdelta/np.sum(expnegdelta,axis=1)
pij=jnp.asarray((pij+pij.T)/2/N)



In [261]:
def KL_scipy(v):
    y=v.reshape(-1,2)
    qij=1/(1+jnp.sum((y[:,None,:]-y[None,:,:])**2,axis=-1))
    qij=qij.at[jnp.diag_indices(N)].set(0.)
    qij = qij/jnp.sum(qij)
    return jnp.sum(pij[nodiagmask]*jnp.log(pij[nodiagmask]/qij[nodiagmask]))

KL_scipy_jit = jax.jit(KL_scipy)
KL_scipy_grad=jax.jit(jax.grad(KL_scipy))

In [269]:
sol=minimize(KL_scipy_jit,y0.flatten(),method='L-BFGS-B', tol=1e-10)
sol

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.2941794693470001
        x: [-1.564e+00  2.165e+00 ...  1.690e+00 -2.692e+00]
      nit: 0
      jac: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
     nfev: 601
     njev: 1
 hess_inv: <600x600 LbfgsInvHessProduct with dtype=float64>

In [256]:
def KL(y):
    qij=1/(1+jnp.sum((y[:,None,:]-y[None,:,:])**2,axis=-1))
    qij=qij.at[jnp.diag_indices(N)].set(0.)
    qij = qij/jnp.sum(qij)
    return jnp.sum(pij[nodiagmask]*jnp.log(pij[nodiagmask]/qij[nodiagmask]))

KLjit = jax.jit(KL)
KLgrad=jax.jit(jax.grad(KL))

In [257]:
y0=jnp.asarray(rng.uniform(0,1,size=(N,2)))
nodiagmask=np.full((N,N),True)
np.fill_diagonal(nodiagmask,False)



In [258]:
gsum=0
yorig=y0
ydefmax=1e-3

In [259]:
#Distance over Gradient - Stochastic Gradient Descent
#https://arxiv.org/pdf/2302.12022.pdf

for i in range(int(5e3)):
    g=KLgrad(y0)
    gsum+=jnp.sum(g**2)
    eta= ydefmax/jnp.sqrt(gsum)
    y1 = y0 - eta*g
    ydelta=jnp.linalg.norm(y1-yorig)
    ydefmax = ydelta if ydelta > ydefmax else ydefmax
    y0=y1
    if i % 1000 == 0:
        print(i, KL(y0), eta)

0 0.69762766 0.06371693
1000 0.30756682 237.94707
2000 0.3059732 209.41904
3000 0.29439437 201.90422
4000 0.29424024 197.19208


In [270]:
df2=pd.DataFrame(y0)
df2=pd.DataFrame(sol.x.reshape(-1,2))
df2['label']=labels
df2['label']=df2['label'].astype('category')

In [271]:
fig2=px.scatter(df2,x=0,y=1,color='label', color_discrete_sequence=px.colors.qualitative.G10)
fig2.update_layout(width=500,height=500,template='plotly_dark')

In [ ]:
y0